In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from webUtils import getHTML
from time import sleep
import time
from multiprocessing import Pool


######################################################
# Music Stuff
######################################################
from dbBase import dbBase
from multiArtist import multiartist
from myMusicDBMap import myMusicDBMap
from mainDB import mainDB
from parseDBArtistsData import parseDBArtistsData




import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Notebook Last Run Initiated: 2020-07-26 00:10:31.146484


In [2]:
%load_ext autoreload
%autoreload
from mainDB import mainDB
mdb    = myMusicDBMap(debug=True)
maindb = mainDB(mdb=mdb, create=True, debug=True)
pdb    = parseDBArtistsData(maindb=maindb)
dbs    = ['AllMusic', 'Discogs', 'MusicBrainz', 'LastFM']
#dbs    = ['LastFM']
for db in dbs:
    pdb.parse(db)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Creating myMusicDBMap()
   Loading my music db map: /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/myMusicMap.p
   DB keys: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
Counter({'Discogs': 3985, 'AllMusic': 3924, 'MusicBrainz': 3722, 'LastFM': 2852, 'DatPiff': 535, 'RockCorner': 479, 'AceBootlegs': 175, 'CDandLP': 172, 'RateYourMusic': 56, 'MusicStack': 5, 'MetalStorm': 0})
Setting Basic Database Objects
=========================   discogs   =========================
=========================   allmusic   =========================
=========================   musicbrainz   =========================
=========================   lastfm   =========================
=========================   rockcorner   =========================
=========================   datpiff   =========================
============

Saving 507 new artist IDs name data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/32-Metadata.p
  --> This file is 23.4kB.
Saving 507 new artist IDs media data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/32-MediaMetadata.p
  --> This file is 283.3kB.
Parsing Artist Files For ModVal 33
  ===> Found 631 files. There are 2 new files to parse.
  ===> Found 634 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 42  ===> Found 630 files, but there are no new files to parse so skipping.

Parsing Artist Files For ModVal 34
  ===> Found 663 files. There are 1 new files to parse.
  ===> Found 563 files. There are 1 new files to parse.
  ===> Found 647 files. There are 1 new files to parse.
Saving 110 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/10-DB.p
Saving 113 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/30-DB.p
  --> This file is 502.4kB.
Saving 532 new artist IDs name data to /Volumes/Piggy/Discog/artists-a

Saving 112 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/64-DB.p
  ===> Found 549 files. There are 2 new files to parse.
  ===> Found 582 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 74
  --> This file is 464.4kB.
Saving 524 new artist IDs name data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/64-Metadata.p
  --> This file is 24.0kB.
Saving 524 new artist IDs media data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/64-MediaMetadata.p
  --> This file is 274.2kB.
Parsing Artist Files For ModVal 65
  ===> Found 655 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 76
Saving 125 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/47-DB.p
  ===> Found 614 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 75
Saving 137 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/35-DB.p
  --> This file is 424.3kB.
Saving 500 new artist I

  --> This file is 255.9kB.
Parsing Artist Files For ModVal 90
  ===> Found 590 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 91
  ===> Found 620 files, but there are no new files to parse so skipping.
Saving 107 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/95-DB.p
  --> This file is 388.9kB.
Saving 481 new artist IDs name data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/95-Metadata.p
  --> This file is 22.5kB.
Saving 481 new artist IDs media data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/95-MediaMetadata.p
  --> This file is 223.9kB.
Saving 124 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/98-DB.p
  --> This file is 452.4kB.
Saving 471 new artist IDs name data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/98-Metadata.p
  --> This file is 21.5kB.
Saving 471 new artist IDs media data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/98-MediaMetadata.p
  --> This file is 273.6kB.
Pars

Saving 7399 new artist IDs name data to /Volumes/Piggy/Discog/artists-discogs-db/metadata/40-Metadata.p
  --> This file is 12.4MB.
Saving 7539 new artist IDs name data to /Volumes/Piggy/Discog/artists-discogs-db/metadata/11-Metadata.p
  --> This file is 12.7MB.
Saving 7498 new artist IDs name data to /Volumes/Piggy/Discog/artists-discogs-db/metadata/15-Metadata.p
  --> This file is 428.9kB.
Saving 7399 new artist IDs media data to /Volumes/Piggy/Discog/artists-discogs-db/metadata/40-MediaMetadata.p
  --> This file is 437.4kB.
Saving 7539 new artist IDs media data to /Volumes/Piggy/Discog/artists-discogs-db/metadata/11-MediaMetadata.p
  --> This file is 432.9kB.
Saving 7498 new artist IDs media data to /Volumes/Piggy/Discog/artists-discogs-db/metadata/15-MediaMetadata.p
  --> This file is 4.0MB.  --> This file is 4.0MB.

  --> This file is 4.1MB.
Parsing Artist Files For ModVal 56
Parsing Artist Files For ModVal 60
Parsing Artist Files For ModVal 41
Saving 1 new artist IDs to /Volumes/P

...

  ===> Found 1289 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 73
  ===> Found 1283 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 80
  ===> Found 1203 files. There are 1 new files to parse.
  ===> Found 1215 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 77
  ===> Found 1225 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 70
  ===> Found 1234 files, but there are no new files to parse so skipping.  ===> Found 1228 files, but there are no new files to parse so skipping.  ===> Found 1239 files, but there are no new files to parse so skipping.


Parsing Artist Files For ModVal 59  ===> Found 1234 files, but there are no new files to parse so skipping.Parsing Artist Files For ModVal 66Parsing Artist Files For ModVal 74



Parsing Artist Files For ModVal 84
  ===> Found 1270 files. There are 1 new files to parse.
Saving 1 new a

  ===> Found 968 files. There are 1 new files to parse.
  ===> Found 962 files. There are 2 new files to parse.
  ===> Found 1001 files. There are 3 new files to parse.
  ===> Found 971 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 29
Saving 1 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/8-DB.pSaving 1 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/0-DB.p
Saving 1 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/20-DB.p

Saving 1 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/24-DB.p
Saving 1 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/12-DB.p
Saving 2 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/4-DB.p
Saving 3 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/16-DB.p
  ===> Found 932 files. There are 1 new files to parse.
  --> This file is 254.2kB.
  --> This file is 255.5kB.
Saving 996 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/2

Saving 1014 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/2-Metadata.p
  --> This file is 271.0kB.  --> This file is 286.8kB.

Saving 996 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/22-Metadata.pSaving 972 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/19-Metadata.p

  --> This file is 65.1kB.
Saving 1014 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/2-MediaMetadata.p
  --> This file is 66.6kB.
Saving 996 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/22-MediaMetadata.p
  --> This file is 63.3kB.
Saving 972 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/19-MediaMetadata.p
  --> This file is 116.0kB.
Parsing Artist Files For ModVal 3  --> This file is 105.3kB.

  --> This file is 122.7kB.
Parsing Artist Files For ModVal 23
Parsing Artist Files For ModVal 44
  ===> Found 929 files. There are 2 new files

Saving 984 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/61-Metadata.pSaving 989 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/46-Metadata.pSaving 962 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/34-Metadata.p
Saving 978 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/50-Metadata.p


  --> This file is 63.3kB.
Saving 970 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/42-MediaMetadata.p
  --> This file is 64.9kB.  --> This file is 62.4kB.

Saving 989 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/46-MediaMetadata.pSaving 962 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/34-MediaMetadata.p

  --> This file is 63.5kB.
  --> This file is 64.7kB.Saving 978 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/50-MediaMetadata.p
  --> This file is 103.3kB

  --> This file is 104.8kB.  --> This file is 91.6kB.Saving 966 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/80-MediaMetadata.p  --> This file is 103.6kB.Saving 968 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/84-MediaMetadata.p




Parsing Artist Files For ModVal 77
Parsing Artist Files For ModVal 96Parsing Artist Files For ModVal 92

  --> This file is 112.2kB.
Parsing Artist Files For ModVal 85
  --> This file is 121.8kB.
Parsing Artist Files For ModVal 81
Saving 1 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/70-DB.p
Saving 1 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/88-DB.p
  --> This file is 269.0kB.
  --> This file is 270.9kB.
Saving 925 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/70-Metadata.pSaving 975 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/88-Metadata.p

  --> This file is 60.5kB.
  --> This file is 63.6kB.Saving 

In [ ]:
nProcs = 8
%load_ext autoreload
%autoreload
from mainDB import mainDB
maindb = mainDB(mdb=None, create=True, debug=True)
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")

parseArtistsParallel(db="AllMusic", nProcs=nProcs)
maindb.setDBFull("AllMusic")

parseArtistsParallel(db="MusicBrainz", nProcs=nProcs)
maindb.setDBFull("MusicBrainz")

parseArtistsParallel(db="Discogs", nProcs=nProcs)
maindb.setDBFull("Discogs")

#parseArtistsParallel(db="AceBootlegs", nProcs=1)
#parseArtistsParallel(db="DatPiff", nProcs=1)
#parseArtistsParallel(db="RateYourMusic", nProcs=3)
parseArtistsParallel(db="LastFM", nProcs=nProcs)
maindb.setDBFull("LastFM")

#parseArtistsParallel(db="RockCorner", nProcs=3)
#parseArtistsParallel(db="CDandLP", nProcs=3)
#parseArtistsParallel(db="MusicStack", nProcs=1)
elapsed(start2, cmt2)
elapsed(start, cmt)

# Master DB

In [ ]:

print("\n\n{0}\n".format("Full DB"))
print("\n\n{0}\n".format("Known DB"))
#maindb.setDBKnown() ## Do this to recreate everything
#maindb.setDBKnown()
#artistDBs = maindb.getKnownArtistDBs()

# Download My Known Artists

In [ ]:
mdb = myMusicDBMap(debug=True)

In [ ]:
dbsToGet = ["AllMusic", "Discogs", "MusicBrainz", "LastFM", "RockCorner", "CDandLP"]
searches = [dbdata["AllMusic"]["Artists"].searchAllMusicForArtist, dbdata["Discogs"]["Artists"].searchDiscogForArtist,
            dbdata["MusicBrainz"]["Artists"].searchMusicBrainzForArtist, dbdata["LastFM"]["Artists"].searchLastFMForArtist,
            dbdata["RockCorner"]["Artists"].searchRockCornerForArtist, dbdata["CDandLP"]["Artists"].searchCDandLPForArtist]

dbsToGet = ["AllMusic"]
searches = [dbdata["AllMusic"]["Artists"].searchAllMusicForArtist]

In [ ]:
def searchAll(mdb, minI=-1):
    nArtists = len(mdb.getArtists())
    for i,artistName in enumerate(mdb.getArtists()):
        if i <= minI:
            continue

        print('\n\n\n')
        print('-'*130)
        print(i,'/',nArtists,'===>\t',artistName)
        print('-'*130)
        print('\n')
        
        dbdata = mdb.getArtistData(artistName)
        if all([dbdata.get("LastFM"), dbdata.get("Discogs"), dbdata.get("AllMusic"), dbdata.get("MusicBrainz")]):
            print("{0} is well known.".format(artistName))
            continue
        else:
            print("{0} get it!".format(artistName))

        for search in searches:
            print("====>",search)
            try:
                search(artistName)
            except:
                sleep(3)

In [ ]:
searchAll(mdb, 3404)

In [ ]:
# Refused

# Albums

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
from discogsBase import discogs

disc = discogs()
albs = albums(disc)
#albs.parseAlbumModValFiles(modVal=27)

In [ ]:
def parseAlbums(modVal):
    albs.parseAlbumModValFiles(modVal=modVal)
    #albs.createAlbumModValMetadata(modVal)
    #albs.createAlbumModValMetadata(modVal=modVal)

def parseAlbumsParallel(modVals=None):
    if modVals is None:
        modVals = ["NAN"] + list(range(100))
        #modVals = range(100)
    nProcs = 2
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, modVals)
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
parseAlbumsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

# Download Stuff

In [ ]:
mdb = myMusicDBMap(debug=True)
maindb = mainDB(mdb=mdb, create=False, debug=True)
dbdata = maindb.dbdata

In [ ]:
ifile = "unmatchedArtists4.p"
noMatches = getFile(ifile)
print(len(noMatches))
dbs = ['Discogs', 'AllMusic', 'MusicBrainz', 'LastFM']
dbs = ['MusicBrainz']

#noMatches=['Go-A', 'PM', 'Qango']

from time import sleep
maxI = -1
for i,artist in enumerate(noMatches):
    print("="*100)
    print("="*100)
    print("="*100)
    print("="*100)
    print(i,'/',len(noMatches),'\t',artist)
    if i <= maxI:
        continue
    for db in dbs:
        print("="*100)
        print(i,'/',len(noMatches),'\t',artist,'===>',db)
        try:
            dbdata[db]["Artists"].searchForArtist(artist)
        except:
            sleep(5)